In [84]:
from autophrase import spacy_txt
import pandas as pd
from snapy import MinHash, LSH
import time
import re

In [3]:
path = "DEA"
s = spacy_txt()
doc, v, n= s.read_file(path)

gg
August 06, 2020
August 06, 2020 DATE
Contact
Phone Number
IMMEDIATE RELEASE
fentanyl-related overdose deaths
officials
public warning
Counterfeit Pills
Fentanyl
Counterfeit Pills
Fentanyl
SAN DIEGO
Law enforcement officials
the county
the public
a sharp increase
overdose deaths
the highly potent and often deadly drug
fentanyl
SAN DIEGO ORG
152 fentanyl-related overdose deaths
San Diego County
the first six months of this year
203 fentanyl-related deaths
pending confirmation
This year
deaths
all areas
the county
Victims
the average age
2019 DATE
San Diego County GPE
the first six months of this year DATE
119 CARDINAL
another 84 CARDINAL
This year DATE
age from 17 DATE
37 DATE
fentanyl-laced drugs
the DEA
San Diego County District Attorney
U.S. Attorney
San Diego County Sheriff’s Department
San Diego Police Department
San Diego County Health and Human Services
San Diego County Medical Examiner
the DEA ORG
San Diego County Sheriff’s Department ORG
San Diego Police Department ORG
San Di

In [4]:
noun = doc[0].sentences[0].noun
ner = doc[0].sentences[0].ner

spacy_dic = {}
for ls in ner:
    for n in ls:
        spacy_dic.update(n)

In [6]:
mesh_fp = '/teams/DSC180A_FA20_A00/a07opioidoverdoseprevalence/data/MESH.csv'
rxnorm_fp = '/teams/DSC180A_FA20_A00/a07opioidoverdoseprevalence/data/RXNORM.csv'
rx=pd.read_csv(rxnorm_fp,usecols=['Preferred Label','Semantic type UMLS property'])
mesh=pd.read_csv(mesh_fp,usecols=['Preferred Label','Semantic type UMLS property'])
term = pd.concat([rx,mesh])
term['Preferred Label'] = term['Preferred Label'].str.lower()

In [7]:
ontology = dict(zip(term['Preferred Label'],term['Semantic type UMLS property']))

In [92]:
terms = list(ontology.keys())
terms=terms

In [93]:
def make_content_all(search_terms, nouns):
    #Initialize a dictionary
    content = dict()
    
    #Add search terms to dictionary with numbered key
    for i in range(len(search_terms)):
        if len(search_terms[i]) > 3:
            content[i] = search_terms[i].strip().lower()
                
    #Add text to dictionary with author as key
    for i in range(len(nouns)):
        key_n = 'sentence%d' % (i)
        count = 0
        for n in nouns[i]:
            n = n.lower()
            if len(n) > 3:
                key = key_n+'_'+str(count)
                content[key] = n
                count += 1
                
    return content

In [94]:
content = make_content_all(terms, noun)

In [95]:
def create_lsh(content, n_permutations, n_gram):
    labels = content.keys()
    values = content.values()
    #Create MinHash object
    minhash = MinHash(values, n_gram=n_gram, permutations=n_permutations, hash_bits=64, seed=3)
    #Create LSH model
    lsh = LSH(minhash, labels, no_of_bands=5)
    return lsh


In [96]:
n_permutations= 50,
n_gram= 3
%time lsh = create_lsh(content,n_permutations,n_gram)

CPU times: user 6min 53s, sys: 3.56 s, total: 6min 57s
Wall time: 6min 56s


In [ ]:
edge_list = lsh.edge_list(min_jaccard=0,jaccard_weighted=True)
similar_ls = []
for i in edge_list:
    if type(i[0])==str and type(i[1])==int:
        similar_ls.append(i)
    elif (type(i[0])==int and type(i[1])==str):
        similar_ls.append(i)

In [ ]:
similar_ls

In [ ]:
entities = copy.deepcopy(ner)
for i in similar_ls:
    word_id = i[0]
    term_id = i[1]
    score = i[2]
    #get the entity 
    word = content[word_id]
    #get the sentence and word index of the entity
    index = word_id.replace('sentence','')
    sent_index = int(re.sub('.+(?=_)','',index).replace('_',''))
    word_index = int(re.sub('(?<=_).+','',index).replace('_',''))
    
    types = []
    scores = []
    # get ontology type for entity
    print(terms[term_id])
    onto = ontology[terms[term_id]]
    types.append(onto)
    scores.append(score)
    
    #check if the noun is already in ner list
    in_ner =False
    ner_score =1
    #loop through list of entity dictionary 
    for n in entities[sent_index]:
        if word in n:
            types.append(n[word])
            scores.append(ner_score)
            n[word] = [types,scores]
            in_ner = True
            
    if not in_ner:
        #store type of entity to its location in the list
        entities[sent_index].append({word:[types,scores]})